In [ ]:
%load_ext autoreload
%autoreload 2

from mimic_iv_analysis.configurations.params import DEFAULT_STUDY_TABLES_LIST
from mimic_iv_analysis.io import DataLoader, TableNames, ParquetConverter, ExampleDataLoader
import pandas as pd


import dask.dataframe as dd

In [2]:
# len( list(set.intersection(*a[:4])) )
examples_ful_w_filtering = ExampleDataLoader(partial_loading=False)

[16:28:52] INFO     Loading subject IDs from admissions table (this will be cached)              data_loader.py:307

[16:28:54] INFO     Loading full table: 546,028 rows.                                            data_loader.py:480

[16:28:56] INFO     Applied filters: 534,159 rows.                                               data_loader.py:484

           INFO     Loading subject IDs from diagnoses_icd table (this will be cached)           data_loader.py:307

           INFO     Loading subject IDs from diagnoses_icd table (this will be cached)           data_loader.py:307

...

           INFO     Loading subject IDs from diagnoses_icd table (this will be cached)           data_loader.py:307

           INFO     Loading subject IDs from diagnoses_icd table (this will be cached)           data_loader.py:307

--- Logging error ---
Traceback (most recent call last):
  File "/Users/artinmajdi/Documents/GitHubs/RAP/mimic__pankaj/mimic_iv_code/.venv/lib/python3.12/site-packages/rich/logging.py", line 178, in emit
    self.console.print(log_renderable)
  File "/Users/artinmajdi/Documents/GitHubs/RAP/mimic__pankaj/mimic_iv_code/.venv/lib/python3.12/site-packages/rich/console.py", line 1719, in print
    extend(render(renderable, render_options))
  File "/Users/artinmajdi/Documents/GitHubs/RAP/mimic__pankaj/mimic_iv_code/.venv/lib/python3.12/site-packages/rich/console.py", line 1340, in render
    for render_output in iter_render:
                         ^^^^^^^^^^^
  File "/Users/artinmajdi/Documents/GitHubs/RAP/mimic__pankaj/mimic_iv_code/.venv/lib/python3.12/site-packages/rich/table.py", line 515, in __rich_console__
    yield from self._render(console, render_options, widths)
  File "/Users/artinmajdi/Documents/GitHubs/RAP/mimic__pankaj/mimic_iv_code/.venv/lib/python3.12/site-packages/rich/ta

Unexpected exception formatting exception. Falling back to standard exception

RecursionError: maximum recursion depth exceeded

In [ ]:
# from dask.distributed import Client
# client = Client(n_workers=4, threads_per_worker=4, processes=True, memory_limit='8GB')
# client

## Scan the directory

In [2]:
from pathlib import Path

MIMIC_DATA_PATH = Path("/Users/artinmajdi/Documents/GitHubs/RAP/mimic__pankaj/dataset/mimic-iv-3.1")

# Initialize DataLoader and scan directory
loader = DataLoader(mimic_path=MIMIC_DATA_PATH)
loader.scan_mimic_directory()



In [3]:
# # Create ParquetConverter
# converter = ParquetConverter(data_loader=loader)

# # Example: Save a small table as Parquet
# converter.save_as_parquet(table_name=TableNamesHOSP.ADMISSIONS)

In [ ]:
# TODO: Load tables individually and merge them with the how=outer (filters applied after merging) and see if it makes any difference compare to when I do it with the inner (filters applied before merging).

examples_full = ExampleDataLoader(partial_loading=False)
# examples_partial = ExampleDataLoader(partial_loading=True, num_subjects=10, random_selection=False)

In [ ]:
examples_partial.study_table_info()

In [ ]:
examples_partial.counter()

In [ ]:
df = examples_partial.load_merged_tables(partial_loading=True, num_subjects=10, random_selection=False, use_dask=True)

In [ ]:
# Print in a tabular format
examples_partial.n_rows_after_merge()